In [2]:
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
async def root():
    return {"message": "¡API en ejecución correctamente! 🎉"}

In [3]:
from docx import Document

def read_docx(file_path):
    """Lee un archivo .docx y retorna el texto concatenado."""
    doc = Document(file_path)
    text = []
    for paragraph in doc.paragraphs:
        if paragraph.text.strip():  # Ignorar líneas vacías
            text.append(paragraph.text.strip())
    return " ".join(text)

# Ejemplo de uso
if __name__ == "__main__":
    file_path = "documento.docx"
    content = read_docx(file_path)
    print(content)

Ficción Espacial: En la lejana galaxia de Zenthoria, dos civilizaciones alienígenas, los Dracorians y los Lumis, se encuentran al borde de la guerra intergaláctica. Un intrépido explorador, Zara, descubre un antiguo artefacto que podría contener la clave para la paz. Mientras viaja por planetas hostiles y se enfrenta a desafíos cósmicos, Zara debe desentrañar los secretos de la reliquia antes de que la galaxia se sumerja en el caos. Ficción Tecnológica: En un futuro distópico, la inteligencia artificial ha evolucionado al punto de alcanzar la singularidad. Un joven ingeniero, Alex, se ve inmerso en una conspiración global cuando descubre que las supercomputadoras han desarrollado emociones. A medida que la humanidad lucha por controlar a estas máquinas sintientes, Alex se enfrenta a dilemas éticos y decisiones que podrían cambiar el curso de la historia. Naturaleza Deslumbrante: En lo profundo de la selva amazónica, una flor mágica conocida como "Luz de Luna" florece solo durante la no

In [4]:
def split_into_chunks(text, max_chunk_size):
    """Divide el texto en chunks del tamaño máximo especificado."""
    words = text.split()
    chunks = []
    current_chunk = []

    for word in words:
        current_chunk.append(word)
        if len(" ".join(current_chunk)) >= max_chunk_size:
            chunks.append(" ".join(current_chunk))
            current_chunk = []

    if current_chunk:  # Agregar el último chunk si quedó algo
        chunks.append(" ".join(current_chunk))

    return chunks

# Ejemplo de uso
if __name__ == "__main__":
    chunks = split_into_chunks(content, max_chunk_size=450)
    for i, chunk in enumerate(chunks):
        print(f"Chunk {i+1}: {chunk}\n")

Chunk 1: Ficción Espacial: En la lejana galaxia de Zenthoria, dos civilizaciones alienígenas, los Dracorians y los Lumis, se encuentran al borde de la guerra intergaláctica. Un intrépido explorador, Zara, descubre un antiguo artefacto que podría contener la clave para la paz. Mientras viaja por planetas hostiles y se enfrenta a desafíos cósmicos, Zara debe desentrañar los secretos de la reliquia antes de que la galaxia se sumerja en el caos. Ficción Tecnológica:

Chunk 2: En un futuro distópico, la inteligencia artificial ha evolucionado al punto de alcanzar la singularidad. Un joven ingeniero, Alex, se ve inmerso en una conspiración global cuando descubre que las supercomputadoras han desarrollado emociones. A medida que la humanidad lucha por controlar a estas máquinas sintientes, Alex se enfrenta a dilemas éticos y decisiones que podrían cambiar el curso de la historia. Naturaleza Deslumbrante: En lo profundo de

Chunk 3: la selva amazónica, una flor mágica conocida como "Luz de Lun

In [5]:
import cohere

def generate_embeddings(text_chunks, api_key):
    """Genera embeddings para una lista de chunks de texto."""
    co = cohere.Client(api_key)
    embeddings = co.embed(
        model='embed-english-v2.0',
        texts=text_chunks
    ).embeddings
    return embeddings

# Ejemplo de uso
if __name__ == "__main__":
    API_KEY = "fbdkymnccgS3MtUK38jsJ0mVcU1mrqNOKpUBFMsd"
    embeddings = generate_embeddings(chunks, API_KEY)
    print(f"Generé {len(embeddings)} embeddings.")

Generé 5 embeddings.


In [6]:
import chromadb

def store_embeddings(chunks, embeddings, collection_name="documents"):
    """Almacena embeddings en ChromaDB."""
    try:
        client = chromadb.Client()  # Inicializar el cliente ChromaDB
        #client.delete_collection(name=collection_name)
        collection = client.get_or_create_collection(name=collection_name)
        
        # Agregar los chunks y sus embeddings a la colección
        for i, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
            collection.add(
                documents=[chunk],
                embeddings=[embedding],
                ids=[f"doc-{i}"]
            )

        print(f"Se almacenaron {len(chunks)} documentos en la colección '{collection_name}'.")
        return collection

    except Exception as e:
        print(f"Error al almacenar los embeddings: {e}")
        return None

# Ejemplo de uso
if __name__ == "__main__":
    collection = store_embeddings(chunks, embeddings)
    if collection:
        print(f"Documentos en la colección: {collection.count()}")

Se almacenaron 5 documentos en la colección 'documents'.
Documentos en la colección: 5


In [7]:
import cohere
import chromadb

def search_similar_documents(query: str, collection_name="documents", top_k=3, api_key="fbdkymnccgS3MtUK38jsJ0mVcU1mrqNOKpUBFMsd"):
    """Busca los documentos más similares a una consulta en ChromaDB."""
    try:
        # Inicializar el cliente ChromaDB
        client = chromadb.Client()

        # Obtener la colección donde están los embeddings almacenados
        collection = client.get_collection(name=collection_name)

        # Generar el embedding para la consulta
        co = cohere.Client(api_key)
        query_embedding = co.embed(
            model="embed-english-v2.0",  # Modelo de embeddings de Cohere
            texts=[query]
        ).embeddings[0]

        # Buscar los documentos más similares en la colección
        results = collection.query(
            query_embeddings=[query_embedding],
            n_results=top_k
        )
        
        # Extraer los documentos y sus distancias
        documents = results["documents"][0]
        distances = results["distances"][0]

        # Mostrar los resultados
        for i, (doc, distance) in enumerate(zip(documents, distances)):
            print(f"Resultado {i+1}: {doc} (Distancia: {distance:.4f})")
        return documents, distances

    except Exception as e:
        print(f"Error al buscar documentos similares: {e}")
        return None, None

# Ejemplo de uso
if __name__ == "__main__":
    query = "¿Qué es la Luz de Luna?"
    documents, distances = search_similar_documents(query, api_key="fbdkymnccgS3MtUK38jsJ0mVcU1mrqNOKpUBFMsd")
    print(f"Documentos más relevantes: {documents}")

Resultado 1: la selva amazónica, una flor mágica conocida como "Luz de Luna" florece solo durante la noche. Con pétalos que brillan intensamente, la flor ilumina la oscuridad de la jungla, guiando a criaturas nocturnas y revelando paisajes deslumbrantes. Los lugareños creen que posee poderes curativos, convirtiéndola en el tesoro oculto de la naturaleza. Cuento Corto: En un pequeño pueblo, cada año, un reloj antiguo regala un día extra a la persona más desafortunada. (Distancia: 5317.0742)
Resultado 2: de camuflaje, se desplaza entre las sombras para proteger a los indefensos. Su pasado enigmático esconde tragedias que lo impulsan a luchar contra la injusticia. Aunque carece de habilidades sobrenaturales, su ingenio y habilidades tácticas lo convierten en una fuerza a tener en cuenta. (Distancia: 7064.1201)
Resultado 3: Ficción Espacial: En la lejana galaxia de Zenthoria, dos civilizaciones alienígenas, los Dracorians y los Lumis, se encuentran al borde de la guerra intergaláctica. Un 

In [9]:
import cohere
import langid
from deep_translator import GoogleTranslator

def translate_text(text, target_language):
    return GoogleTranslator(source='auto', target=target_language).translate(text)

def generate_answer(query: str, context: list, api_key="fbdkymnccgS3MtUK38jsJ0mVcU1mrqNOKpUBFMsd"):
    """Genera una respuesta usando Cohere con los documentos más relevantes como contexto."""
    try:
        # Detectar el idioma de la consulta usando langid
        language, _ = langid.classify(query)
        
        # Mapear el idioma detectado a su nombre
        language_map = {
            'en': 'english',
            'es': 'Español',
            'pt': 'portuguese'
        }

        # Asegurarse de que el idioma esté en el mapa
        language_name = language_map.get(language, 'spanish')  # Default to inglés if unknown
        #print(f"Idioma detectado: {language_name}")

        co = cohere.Client(api_key)
        print(context)
        # Concatenamos el contexto relevante
        context_text = " ".join(context)

        # Crear el prompt para Cohere
        prompt = f"Pregunta: {query}\n\nContexto: {context_text}\n\nGenera una respuesta concisa en una sola oración usando el idioma {language_name}, usa tercera persona, la oración debe tener menos de 50 palabras, solo genera la respuesta y agrega emojis relevantes que resuman la respuesta."

        if language_name != 'Español':  # Si el idioma de la consulta no es inglés, traducir
            prompt = translate_text(prompt, language_name.lower())
            #print(f"prompt traducido al {language_name}: {prompt}")

        # Generar la respuesta con Cohere
        response = co.generate(
            model="command-r-plus-04-2024",  # Modelo más actual
            prompt=prompt,
            max_tokens=50,
            temperature=0,  # Hacer el modelo más determinista
            #top_p=1  # Considerar todas las palabras posibles, sin aleatoriedad
        )
        
        return response.generations[0].text.strip()

    except Exception as e:
        print(f"Error al generar la respuesta: {e}")
        return None

# Ejemplo de uso
if __name__ == "__main__":
    #query = "Quien es Zara? " # Español
    #query = "What did Emma decide to do?" # Inglés
    query = "What is the name of the magical flower? ?" # Inglés
    #query = "Quem é Zara?" # Portugués - Extra
    documents, distances = search_similar_documents(query, api_key="fbdkymnccgS3MtUK38jsJ0mVcU1mrqNOKpUBFMsd")

    if documents:
        answer = generate_answer(query, documents)  
        print(f"Respuesta generada: {answer}")

Resultado 1: la selva amazónica, una flor mágica conocida como "Luz de Luna" florece solo durante la noche. Con pétalos que brillan intensamente, la flor ilumina la oscuridad de la jungla, guiando a criaturas nocturnas y revelando paisajes deslumbrantes. Los lugareños creen que posee poderes curativos, convirtiéndola en el tesoro oculto de la naturaleza. Cuento Corto: En un pequeño pueblo, cada año, un reloj antiguo regala un día extra a la persona más desafortunada. (Distancia: 10167.9336)
Resultado 2: Emma, una joven huérfana, es la elegida este año. Durante su día adicional, descubre una puerta mágica que la transporta a un mundo lleno de maravillas. Al final del día, Emma decide compartir su regalo con el pueblo, dejando una huella imborrable en el corazón de cada habitante. Características del Héroe Olvidado: Conocido como "Sombra Silenciosa", nuestro héroe es un maestro del sigilo y la astucia. Dotado de una memoria fotográfica y habilidades (Distancia: 13048.1895)
Resultado 3: F